# Particle Swarm Optimization 
## Introduction
The problem:
In this assignment, we will be using the Particle Swarm algorithm to find the global minimum of a polynomial function. Particle Swarm Optimization is a biologically inspired algorithm that mimics a flock of birds or a school of fish. Similar to the Genetic Algorithm, PSO starts with a randomly created initial population, a set of possible solutions (called particles), and updates these particles after every iteration. However, GA uses crossover and mutation to create new individuals, whereas PSO maintains the same population but changes each particle over time. 

We have provided code stubs in this notebook to get you started, and give hints about the structure of the code.

You need to implement Particle Swarm Optimization, and analyze the results. Then, you will qualitatively compare PSO to GA. 

You will need to submit a report along with the finished Jupyter notebook in which you should report the performance of the algorithm. The report instructions are in the bottom section of this notebook.

## Approach: 
Let's start with a few definitions. 
- Particle (aka "individual"): a solution to the polynomial function, which we are trying to minimize. It is specified by an $n$-dimensional vector where $n$ is the dimension of the polynomial ($n$ = 6 for this assignment). The particle's position in the swarm is defined to be this vector, which represents a possible solution to the problem. The particle also has a velocity vector, and variables that record the "best" (lowest) value found so far. 
- Population: a collection of possible solutions to the polynomial (i.e., a collection of Particles)
- Fitness: a function that tells us how good each Particle is (in our case, how low of a value the Particle yields)
-Best Value: another way to talk about the fitness; this is defined as the Particle's lowest value found so far, from plugging in a position into $f(\overrightarrow{x})$

Our Particle Swarm will proceed in the following steps:
1. Create the population (initialized with random positions and velocities)
2. Initialize the global best value and all the Particles' individual best values. All best value initializations should be to $+\infty$
2. Update each Particle's individual best as well as the global best value found so far 
3. Update each Particle's velocity subject to the equation learned in class, and then update the Particle's position 
4. Update $w$
5. Repeat steps 2-4 for 100 iterations
6. Plot your results (best values)

### Algorithm:
At every iteration, the position of each Particle gets updated. Each Particle has a position vector, $x$, and a velocity vector, $v$. The velocity vector is changed, and then each element of the velocity vector gets added to each element of the position vector. The length of $x$ and $v$ is equal to dimension ($n$).

The velocity vector is updated in the following way: 

$v_i^{t+1}$ = inertia + cognitive velocity + social velocity 

- inertia = $w$ * $v_i^{t}$

- cognitive velocity = $c_1$ * $r_1$ * (personal best position - current position)

- social velocity = $c_2$ * $r_2$ * (global best position - current position)

And the position vector is then updated:

$x_i^{t+1}$ = $x_i^{t}$ + $v_i^{t+1}$

$r_1$ and $r_2$ are random numbers between 0 and 1, and $c_1$ and $c_2$ are the cognitive and social parameters, respectively.


The direction that the particle will move in each iteration depends on these three vectors (inertia, cognitive velocity, social velocity). Cognitive velocity pushes the Particle towards its personal best, and social (global) velocity pushes the Particle towards the global best. 

![picture](https://drive.google.com/uc?export=view&id=1O_fB3TsyyTsFDHVjBy99GIQ14tWSzL_n)


In [ ]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt
import operator
import math
import sys
from datetime import datetime

We will set global variables for dimension, degree, and coefficients. Coefficients contains the scalars in the function polynomial, dimension is equal to the number of variables in the polynomial, and degree is the degree of the polynomial. The function polynomial we are trying to minimize is: 

$f(\overrightarrow{x}) = (Ax_1^6 + Bx_2^5 + Cx_3^4 + Dx_4^3 + Fx_5^2 + Gx_6 + H)^8$

The function is written in Python below. We will be using one fixed set of coefficients for this assignment.

In [1]:
dim = 6
degree = 8
coefficients = [50.9428194161253,
                55.14119447504069,
                -38.28266879248081,
                -35.18573732572916,
                16.10442493713063,
                39.28095016666805,
                44.12232630669219]

In [2]:
def func(x, coefficients):
    res = 0
    for i in range(dim):
        res += coefficients[i]*(x[i]**(dim-i))
    res += coefficients[-1]
    return (res)**degree 

## Create necessary classes and functions
Next, we create a class that will represent and handle an individual Particle in the population. A Particle only needs its position vector to be initialized. Velocity starts with random values, and we have variables that keep track of this Particle's best position so far and the value that this position yields. This means that this is the position, or solution, found by this Particle that gives the lowest value when plugged into the polynomial. 

Fitness is simply the value of the polynomial when we plug in this Particle's position. 

There are functions that are used to update the velocity and position of the Particle based on the equations above. When updating position, we must make sure each element of position stays within the desired bounds. This means that if the new position value is outside of the interval [$-100, 100$], we set the value to the boundary value. 

In [ ]:
class Particle:
    def __init__(self, position):
        """
        position: the position of this Particle, a list of length 6
        velocity: the velocity of this Particle, a list of length 6
        best_position: the best position found so far by this Particle
        best_value: the best (lowest) value found so far by this Particle
        """
        self.position = position
        self.velocity = [random.uniform(-5,5) for i in range(dim)]
        self.best_position = np.copy(position)
        self.best_value = math.inf
    
    def fitness(self):
        """
        This fitness function is defined as the value of this Particle
        when plugged into the function polynomial.
        """
        return func(self.position, coefficients) 

    def update_velocity(self, w, c1, c2, best_position_global):
        """
        w, c1, c2: constants
        best_position_global: the best position found in the whole population so far 
        This function updates the Particle's velocity
        """
        # YOUR CODE HERE

    def update_position(self):
        """
        Updates the Particle's position
        """
        # YOUR CODE HERE 

Next, we create the PSO_Simulation class. This will start and run the swarm for the algorithm. We create a PSO_Simulation object with initial population, which is the list of Particles that will be used, and the constants. 

There is a function used to update the best values of a Particle, and update the global best if necessary. 

The main driver of the program is PSO_Simulation.run(), which first updates the best values for each Particle. Then, we begin the iterations. In each iteration, we go through every Particle, update its velocity, update its position, and update its best value if necessary. At the end of each iteration, we update $w$ (set $w = 0.98 * w$), and record the current global best value (lowest value). This function then returns a list of the global bests after each iteration. 

In [ ]:
class PSO_Simulation:
    def __init__(self, initialPop, w, c1, c2):
        """
        initialPop: list of pop_size Particles
        w, c1, c2: constants
        best_position_global: the best position found by any Particle so far
        best_value_global: the best value found by any Particle so far 
        """
        self.particles = initialPop
        self.pop_size = len(initialPop)
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.best_position_global = np.copy(initialPop[0].position)
        self.best_value_global = math.inf

    def update_best(self, particle):

        # YOUR CODE HERE

    def run(self, iterations):

        for i in range(self.pop_size):
            self.update_best(self.particles[i])
    
        # YOUR CODE HERE 
        
        return []
    

## Create our initial population
Particle generator. 
We now can make our initial population. To do so, we need a function that produces random Particles. To create an individual, we randomly generate a position vector, setting each element equal to a random number between $-30$ and $30$. Even though we are starting out with a completely random initial population, there is still a chance for convergence. 

The first function here produces one random Particle, and in the next function, we create the whole initial population by repeatedly calling randomParticle(). Note: we only have to use these functions to create the initial population.

In [ ]:
def randomParticle():

    # YOUR CODE HERE
    
    return None

def initialPopulation(pop_size):
    """
    Create inital population of a given size.
    Returns a list of random Particles
    """
    pop = []
    for i in range(pop_size):
        temp = randomParticle()
        pop.append(temp)
    
    return pop 

## Final Step
The particleSwarm function puts everything together. We start off by creating the initial population, then create a PSO_Simulation object, which we use to run the program. You should capture the initial best value, final best value, and also the best values over time. We then plot the best value found at every iteration, using the list returned by PSO_Simulation.run(). Take the log of each value in the list before plotting. To take the log of a value, use `math.log1p(value)`. This allows for taking the log even when the value is very close to zero. 

In [ ]:
def particleSwarm(iterations, pop_size, w, c1, c2):
    start = datetime.now()

    # YOUR CODE HERE

    end = datetime.now()
    print("Time Elapsed for Particle Swarm: " + str(end - start))


Finally, call the function with your desired parameters. Please choose what parameters to use which give you good results. We recommend keeping pop_size over 200, $w$ under 1, and choosing small values for $c_1$ and $c_2$. Experiment with different combinations of parameters to see what works best.

In [ ]:
particleSwarm(iterations=, pop_size=, w=, c1=, c2=)

# Report and Submission Guidelines

The purpose of this assignment is to analyze the performance of the Particle Swarm Optimization Algorithm. 

The final report should contain a plot that shows how the log of the global "best value" has changed over time. Additionally, you should discuss your results. Then, discuss the similarities and differences between Particle Swarm Optimization and Genetic Algorithm. 

Please submit your code along with the report.

## Extra Credit
See if you can code a visualization of the movement of the particles as the iterations progress. Sort of like an animation of the particles swarming around.
